In [65]:
from multiprocessing.managers import SharedMemoryManager
from multiprocessing import Pool
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio import SeqIO
from Bio.SeqIO.FastaIO import FastaTwoLineIterator

def count_kmers(seq):
    KMERLEN=31
    max_index = len(seq) - KMERLEN + 1
    matched_kmer_strains = []
    with memoryview(bytes(seq)) as seq_buffer:
        kmerset = { seq[i:i+KMERLEN] for i in range(max_index) if i < max_index}
    return kmerset


# fastq_generator = (j.seq for j in FastaTwoLineIterator('/home/mladen/Strainr/inputs/genomes/GCF_000011745.1_ASM1174v1_genomic.fna'))
# fastq_generator = (j.seq for j in SeqIO.parse('/home/mladen/Strainr/inputs/genomes/GCF_000011745.1_ASM1174v1_genomic.fna','fasta')
ii = SeqIO.parse('/home/mladen/Strainr/inputs/genomes/GCF_000011745.1_ASM1174v1_genomic.fna','fasta')
ii = next(ii)
ii.seq
print(len(count_kmers(ii.seq)))

# with SharedMemoryManager() as smm:
#     sl = smm.ShareableList(list(fastq_generator))
    # p = Pool()
    # results = p.map(count_kmers,sl)

# [kmer.tobytes() for kmer in count_kmers(sl[0])]
# len(list(results))


791208


In [ ]:

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib 
x=[1,2,3,900]
plt.scatter(x=x,y=x)

In [11]:
# import libraries
from statsmodels.base.model import GenericLikelihoodModel
import statsmodels.api as sm
import numdifftools as ndt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import scipy.stats as stats

import pymc3 as pm3


In [12]:
# generate data
N = 100
x = np.linspace(0,20,N)
ϵ = np.random.normal(loc = 0.0, scale = 5.0, size = N)
y = 3*x + ϵ
df = pd.DataFrame({'y':y, 'x':x})
df['constant'] = 1
df

,y,x,constant
0,1.937633,0.000000,1
1,1.773369,0.202020,1
2,8.729788,0.404040,1
3,-2.440930,0.606061,1
4,-4.085122,0.808081,1
...,...,...,...
95,55.071758,19.191919,1
96,52.711686,19.393939,1
97,59.426735,19.595960,1
98,65.350304,19.797980,1


In [15]:
sns.regplot(x=df.x, y=df.y);

In [17]:
# split features and target
X = df[['constant', 'x']]# fit model and summarize
sm.OLS(y,X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     1195.
Date:                Sun, 06 Jun 2021   Prob (F-statistic):           1.04e-56
Time:                        04:52:19   Log-Likelihood:                -306.30
No. Observations:                 100   AIC:                             616.6
Df Residuals:                      98   BIC:                             621.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant      -1.0166      1.038     -0.979      0.330      -3.076       1.043
x              3.1001      0.090     34.574      0.000       2.922       3.278
==============================================================================
Omnibus:                        3.015   Durbin-Watson:                   2.232
Prob(Omnibus):                  0.222   Jarque-Bera (JB):                2.625
Skew:                           0.296   Prob(JB):                        0.269
Kurtosis:                       2.471   Cond. No.                         23.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# define likelihood function
def MLERegression(params):
 intercept, beta, sd = params[0], params[1], params[2] # inputs are guesses at our parameters
 yhat = intercept + beta*x # predictions# next, we flip the Bayesian question
# compute PDF of observed values normally distributed around mean (yhat)
# with a standard deviation of sd
 negLL = -np.sum( stats.norm.logpdf(y, loc=yhat, scale=sd) )# return negative LL
 return(negLL)

In [24]:
# let’s start with some random coefficient guesses and optimize
guess = np.array([5,5,2])
results = minimize(MLERegression, guess, method = 'Nelder-Mead', options={'disp': True})
results

Optimization terminated successfully.
         Current function value: 306.302761
         Iterations: 112
         Function evaluations: 196


 final_simplex: (array([[-1.016593  ,  3.10006921,  5.17629486],
       [-1.01665829,  3.10007342,  5.17627617],
       [-1.01658948,  3.10006534,  5.17625677],
       [-1.01656235,  3.1000683 ,  5.1762672 ]]), array([306.30276131, 306.30276131, 306.30276132, 306.30276132]))
           fun: 306.3027613121705
       message: 'Optimization terminated successfully.'
          nfev: 196
           nit: 112
        status: 0
       success: True
             x: array([-1.016593  ,  3.10006921,  5.17629486])

In [26]:
resultsdf = pd.DataFrame({'coef':results['x']})
resultsdf.index=['constant','x','sigma']
np.round(resultsdf.head(2), 4)

,coef
constant,-1.0166
x,3.1001


In [22]:
import ncbi_genome_download as ngd

ngd.download(
    groups="bacteria",
    file_formats="fasta",
    assembly_levels='complete',
    # taxids=727,
    section="genbank",
    assembly_accessions="GCA_000009005.1",
    output="out",
    # parallel=8,
    metadata_table='hi.tsv',
)
# ngd.download??


ValueError: Unrecognized option(s): dict_keys(['verbose'])

In [ ]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

ModuleNotFoundError: No module named 'ete3'